# Aplicação: Detecção de uso de máscaras
Os passos para aplicar nosso classificador são os seguintes:

1.   Carregar modelo treinado
2.   Obter cada frame do video
2.   Detecção de rostos em cada frame
3.   Extração de cada rosto
4.   Aplicar o classificador
5.   Mostrar a classe prevista no frame
6.   Salvar frame no video

Verificar que o tipo de ambiente de execução é **GPU**.
Para mudar o tipo de ambiente de execução, ir para:

`Ambiente de execução -> Alterar tipo de ambiente de execução`

Em Acelerador de hardware, selecionar **GPU**

## Montar o Drive

Executando as seguinte linhas de codigo, podemos montar nosso Google Drive no Colab:

```python
from google.colab import drive
drive.mount('/content/drive')
```
Depois, um link sera apresentado onde temos que accesar para obter um token. Finalmente temos que colar o token nosso Colab para permitir o acceso ao nossos arquivos.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My\ Drive/Colabs/Mask_Detection

/content/drive/My Drive/Colabs/Mask_Detection


## Instalar Dependencias

Neste caso, só vamos precisar da biblioteca [face-recognition](https://pypi.org/project/face-recognition/) para fazer a detecção de rostos e extrair os marcos faciais.


```
!pip install -U face-recognition
```


In [3]:
!pip install -U face-recognition

Requirement already up-to-date: face-recognition in /usr/local/lib/python3.6/dist-packages (1.3.0)


## Import das bibliotecas necessarias

In [4]:
import numpy as np
import cv2
import face_recognition
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook

## Código Principal

Definimos os parâmetros fixos, neste caso são o nome do video a ser processado,
```python
input_video_filename = "test_video.mp4"
```
o nome do video de saida,
```python
output_video_filename = "test_video_output.mp4"
```
o tamanho dos frames,
```python
frame_size = (640,480)
```
e o modelo treinado, o qual carregamos com a função `load_model`
```python
model = load_model("model.h5")
```



In [5]:
input_video_filename = "test_video.mp4"
output_video_filename = "test_video_output.mp4"
frame_size = (640,480)
model = load_model("model.h5")

Agora criamos o objeto para escrever o video processado. Temos que definir o tipo de codec a ser usado, os `frames-per-second` e o tamanho do frame:
```python
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps=20.0
out = cv2.VideoWriter(output_video_filename,fourcc, fps, frame_size)
```



In [6]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps=20.0
out = cv2.VideoWriter(output_video_filename,fourcc, fps, frame_size)

Inicializamos o objeto para a leitura de cada frame

```python
cap = cv2.VideoCapture(input_video_filename)
```
e obtemos a quantidade total de frames no video


```python
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))1
```




In [7]:
cap = cv2.VideoCapture(input_video_filename)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

### Processando cada frame

Começamos com um loop para obter cada frame do video:
```python
while (cap.isOpened()):
    ret, frame = cap.read()
```
Se o frame é valido, fazemos a conversão para RGB, mudamos o tamanho do frame e obtemos a localização de cada rosto na imagem:
```python    
    if ret:
      frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)      
      frame = cv2.resize(frame, frame_size)
      face_locations = face_recognition.face_locations(frame, model="cnn")
```
Agora só se tiver pelo menos um rosto na imagem, iteramos sobre os rostos e extraimos as coordenadas do rosto:
```python 
      if len(face_locations) is not 0:
        for face_location in face_locations:
          top, right, bottom, left = face_location
```
Extraimos o rosto da imagem e fazemos um resize para (224,224) que é o tamanho das imagens com as quais o modelo foi treinado:
```python
          face_image = frame[top:bottom, left:right]
          face_image = cv2.resize(face_image,(224,224))
```
Pre-processamos o rosto e acrescentamos uma dimensão para fazer a previsão usando o modelo treinado:
```python
          face_image = preprocess_input(face_image)
          face_image = np.expand_dims(face_image, axis=0)

          (mask, withoutMask) = model.predict(face_image)[0]
```
O resultado são as probabilidades de ter ou não ter máscara. Definimos o cor verde `(0,255,0)` para os que tem máscara, e vermelho `(255,0,0)` para os casos que não tem máscara
```python
          label = "Mask" if mask > withoutMask else "No Mask"
          color = (0, 255, 0) if label == "Mask" else (255, 0, 0)
```
Mostramos um texto contendo a etiqueta prevista e a probabilidade. Também desenhamos um retângulo na localização do rosto
```python
          # include the probability in the label
          label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
          # display the label and bounding box rectangle on the output frame
          cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
          cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
```
Finalmente, escrevemos cada frame e se não tiver mais frames, saímos do loop principal e liberamos os objetos para leitura e escrita do video.
```python
      out.write(frame)
    else:
      break

out.release()
cap.release()
```



In [8]:
pbar = tqdm_notebook(total=total_frames, desc="Processing video:")

while (cap.isOpened()):
    ret, frame = cap.read()
    pbar.update(1)

    if ret:
      frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)      
      frame = cv2.resize(frame, frame_size)
      face_locations = face_recognition.face_locations(frame, model="cnn")
      if len(face_locations) is not 0:
        for face_location in face_locations:
          top, right, bottom, left = face_location
          face_image = frame[top:bottom, left:right]
          face_image = cv2.resize(face_image,(224,224))        
          face_image = preprocess_input(face_image)
          face_image = np.expand_dims(face_image, axis=0)

          (mask, withoutMask) = model.predict(face_image)[0]

          label = "Mask" if mask > withoutMask else "No Mask"
          color = (0, 255, 0) if label == "Mask" else (255, 0, 0)
          # include the probability in the label
          label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
          # display the label and bounding box rectangle on the output frame
          cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
          cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
      frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)      
      out.write(frame)
    else:
      break

pbar.close()
out.release()
cap.release()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.
